# Prototype Code for Delaney

## 1. Import libraries and declare file name

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sflx_file = '8_3_23_LDP1_DB_Total_Sap_Flow.csv'

## 2. Open file and clean data

In [2]:
sflx_df = pd.read_csv(sflx_file)
sflx_df

,Date/Time,Total Sap Flow (L/hr or kg/hr)
0,5/25/2023 17:00,0.44
1,5/25/2023 18:00,0.42
2,5/25/2023 19:00,0.40
3,5/25/2023 20:00,0.35
4,5/25/2023 21:00,0.25
...,...,...
35149,1/0/1900 0:00,NaN
35150,1/0/1900 0:00,NaN
35151,1/0/1900 0:00,NaN
35152,1/0/1900 0:00,NaN


#### 2a. Report on data types

In [3]:
sflx_df.dtypes

Date/Time                          object
Total Sap Flow (L/hr or kg/hr)    float64
dtype: object

#### 2b. Get rid of rows with dates that don't make sense

In [5]:
sflx_df_filt = sflx_df.drop(sflx_df['Date/Time'] != '1/0/1900 0:00')
sflx_df_filt

KeyError: '[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True

#### 2c. Check datatypes again

In [ ]:
sflx_df_filt.dtypes

#### 2d. Convert object to datetime object and verify

In [ ]:
sflx_df_filt['Date/Time'] = pd.to_datetime(sflx_df_filt['Date/Time'])
sflx_df_filt.dtypes

#### 2e. Set index to the `Date/Time` column

In [ ]:
sflx_df_filt.set_index('Date/Time',inplace=True)
sflx_df_filt

## 3. Get diurnal cycles by using groupby on index hour

#### 3a. Get the average diurnal cycle

In [ ]:
diurnal_mean = sflx_df_filt.groupby(sflx_df_filt.index.hour).mean()
diurnal_mean

#### 3b. Get the standard deviation in the diurnal cycle

In [ ]:
diurnal_std = sflx_df_filt.groupby(sflx_df_filt.index.hour).std()
diurnal_std

#### 3c. Now create and label a plot and save a `.png`

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(diurnal_mean.index,diurnal_mean['Total Sap Flow (L/hr or kg/hr)'],'k',label='$\mu$')
plt.plot(diurnal_mean.index,diurnal_mean['Total Sap Flow (L/hr or kg/hr)']+diurnal_std['Total Sap Flow (L/hr or kg/hr)'],'r',label='$\mu + \sigma$')
plt.plot(diurnal_mean.index,diurnal_mean['Total Sap Flow (L/hr or kg/hr)']-diurnal_std['Total Sap Flow (L/hr or kg/hr)'],'r',label='$\mu - \sigma$')
plt.xlabel('Hour')
plt.ylabel('Total Sap Flow (L/hr)')
plt.legend()
plt.savefig('sapflux_diurnal_all.png')
plt.show()

## 4. A slightly more complicated value

Here we're going to group by both month and hour and compute the standard deviation. The result in a new dataframe with a `MultiIndex` with months and hours of the day. We'll need to be careful in how we access that but, in principle, we can create a plot of the diurnal cycle and how it varies by month.

In [ ]:
sflx_mon_diel_mean = sflx_df_filt.groupby([sflx_df_filt.index.month,sflx_df_filt.index.hour]).mean()
sflx_mon_diel_mean

In [ ]:
sflx_mon_diel_std = sflx_df_filt.groupby([sflx_df_filt.index.month,sflx_df_filt.index.hour]).std()
sflx_mon_diel_std

#### 4a. How do we use the `MultiIndex`?

The `MultiIndex` is a tuple ordered by (month,hour). We can call a single month by calling `.loc(month,)` which will get all hours for a month. For instance, here's the average diurnal cycle for June:

In [ ]:
sflx_mon_diel_mean.loc[6,]

#### 4b. Now create a plot of the average and average ± standard deviation sapflux for each month, save a figure

In [ ]:
plt.figure(figsize=(16,4))

# Plot May
plt.subplot(1,4,1)
plt.title('May')
plt.plot(sflx_mon_diel_mean.loc[5,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[5,],'k')
plt.plot(sflx_mon_diel_mean.loc[5,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[5,]+sflx_mon_diel_std['Total Sap Flow (L/hr or kg/hr)'].loc[5,],'r')
plt.plot(sflx_mon_diel_mean.loc[5,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[5,]-sflx_mon_diel_std['Total Sap Flow (L/hr or kg/hr)'].loc[5,],'r')
plt.ylabel('Total Sap Flow (L/hr)')
plt.xlabel('Hour')
plt.ylim([-0.1,0.8])
plt.legend(['$\mu$','$\mu + \sigma$','$\mu - \sigma$'])

# Plot June
plt.subplot(1,4,2)
plt.title('June')
plt.plot(sflx_mon_diel_mean.loc[6,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[6,],'k')
plt.plot(sflx_mon_diel_mean.loc[6,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[6,]+sflx_mon_diel_std['Total Sap Flow (L/hr or kg/hr)'].loc[6,],'r')
plt.plot(sflx_mon_diel_mean.loc[6,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[6,]-sflx_mon_diel_std['Total Sap Flow (L/hr or kg/hr)'].loc[6,],'r')
plt.xlabel('Hour')
plt.ylim([-0.1,0.8])

# Plot July
plt.subplot(1,4,3)
plt.title('July')
plt.plot(sflx_mon_diel_mean.loc[7,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[7,],'k')
plt.plot(sflx_mon_diel_mean.loc[7,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[7,]+sflx_mon_diel_std['Total Sap Flow (L/hr or kg/hr)'].loc[7,],'r')
plt.plot(sflx_mon_diel_mean.loc[7,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[7,]-sflx_mon_diel_std['Total Sap Flow (L/hr or kg/hr)'].loc[7,],'r')
plt.xlabel('Hour')
plt.ylim([-0.1,0.8])

# Plot August
plt.subplot(1,4,4)
plt.title('August')
plt.plot(sflx_mon_diel_mean.loc[8,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[8,],'k')
plt.plot(sflx_mon_diel_mean.loc[8,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[8,]+sflx_mon_diel_std['Total Sap Flow (L/hr or kg/hr)'].loc[8,],'r')
plt.plot(sflx_mon_diel_mean.loc[8,].index,sflx_mon_diel_mean['Total Sap Flow (L/hr or kg/hr)'].loc[8,]-sflx_mon_diel_std['Total Sap Flow (L/hr or kg/hr)'].loc[8,],'r')
plt.xlabel('Hour')
plt.ylim([-0.1,0.8])

plt.savefig('sapflux_diurnal_by_month.png')
plt.show()